### Create and train the network to predict steering angle for car in simulator

#### Read the training data and store images and steering measurements in to np arrays

In [6]:
import csv
import numpy as np
import matplotlib.image as mpimg

lines = []
csvfile = open('../P3_Support/Training_Data/t1/driving_log.csv')
reader = csv.reader(csvfile)
for line in reader:
    lines.append(line)
    
cimages = []
steerings = []

for line in lines:
    orig_path = line[0]
    new_base_path = '../P3_Support/Training_Data/t1/IMG/'
    cfile_name = orig_path.split('\\')[-1] 
    cfile_path = new_base_path + cfile_name
    image = mpimg.imread(cfile_path)
    cimages.append(image)
    steer = float(line[3])
    steerings.append(steer)

lines = []
csvfile = open('../P3_Support/Training_Data/t1_recovery_left/driving_log.csv')
reader = csv.reader(csvfile)
for line in reader:
    lines.append(line)
    
for line in lines:
    orig_path = line[0]
    new_base_path = '../P3_Support/Training_Data/t1_recovery_left/IMG/'
    cfile_name = orig_path.split('\\')[-1] 
    cfile_path = new_base_path + cfile_name
    image = mpimg.imread(cfile_path)
    cimages.append(image)
    steer = float(line[3])
    steerings.append(steer)
    
lines = []
csvfile = open('../P3_Support/Training_Data/t1_recovery_right/driving_log.csv')
reader = csv.reader(csvfile)
for line in reader:
    lines.append(line)
    
for line in lines:
    orig_path = line[0]
    new_base_path = '../P3_Support/Training_Data/t1_recovery_right/IMG/'
    cfile_name = orig_path.split('\\')[-1] 
    cfile_path = new_base_path + cfile_name
    image = mpimg.imread(cfile_path)
    cimages.append(image)
    steer = float(line[3])
    steerings.append(steer)

lines = []
csvfile = open('../P3_Support/Training_Data/t1_reverse/driving_log.csv')
reader = csv.reader(csvfile)
for line in reader:
    lines.append(line)
    
for line in lines:
    orig_path = line[0]
    new_base_path = '../P3_Support/Training_Data/t1_reverse/IMG/'
    cfile_name = orig_path.split('\\')[-1] 
    cfile_path = new_base_path + cfile_name
    image = mpimg.imread(cfile_path)
    cimages.append(image)
    steer = float(line[3])
    steerings.append(steer)

#Data agumentation
aug_images = []
aug_steerings = []
for image, steering in zip(cimages, steerings):
    aug_images.append(image)
    aug_steerings.append(steering)
    image_flipped = np.fliplr(image)
    steer_flipped = -steering
    aug_images.append(image_flipped)
    aug_steerings.append(steer_flipped)
    
    
X_train = np.array(aug_images)
y_train = np.array(aug_steerings)

print(X_train.shape)
print(y_train.shape)



(7852, 160, 320, 3)
(7852,)


#### Split the training data for validation, construct a network

In [12]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D, Dropout, Cropping2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
#Normalization
model.add(Lambda(lambda x: (x/255.0 - 0.5), input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50, 20), (0, 0))))
#Convolution->relu->MaxPooling
model.add(Conv2D(12, 5, 5, activation='relu'))
model.add(MaxPooling2D())
#Dropout
model.add(Dropout(0.6))
#Convolution->relu->MaxPooling
model.add(Conv2D(6, 5, 5, activation='relu'))
model.add(MaxPooling2D())
#Dropout
model.add(Dropout(0.6))
#Flatten
model.add(Flatten())
#Fully connected
model.add(Dense(120))
model.add(Dropout(0.6))
model.add(Dense(60))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=7)

model.save('model.h5')



Train on 6281 samples, validate on 1571 samples
Epoch 1/7
6281/6281 [==============================] - 280s - loss: 0.5598 - val_loss: 0.0478
Epoch 2/7
6281/6281 [==============================] - 321s - loss: 0.0366 - val_loss: 0.0462
Epoch 3/7
6281/6281 [==============================] - 327s - loss: 0.0339 - val_loss: 0.0461
Epoch 4/7
6281/6281 [==============================] - 305s - loss: 0.0333 - val_loss: 0.0459
Epoch 5/7
6281/6281 [==============================] - 313s - loss: 0.0318 - val_loss: 0.0457
Epoch 6/7
6281/6281 [==============================] - 302s - loss: 0.0311 - val_loss: 0.0459
Epoch 7/7
6281/6281 [==============================] - 313s - loss: 0.0297 - val_loss: 0.0456
